<a href="https://colab.research.google.com/github/pranshu-raj-211/project-alphazero/blob/main/breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

In [2]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from scipy import stats
import plotly.graph_objects as go

df=pd.read_csv('EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv')
df=df[df['volume']!=0]
df.reset_index(drop=True,inplace=True)


df['EMA_150']=ta.ema(df.close,length=150)

df=df[:10000]

In [3]:
print(df.tail())

                     Gmt time     open     high      low    close  \
9995  07.12.2004 17:00:00.000  1.34357  1.34367  1.34151  1.34221   
9996  07.12.2004 18:00:00.000  1.34233  1.34341  1.34168  1.34227   
9997  07.12.2004 19:00:00.000  1.34215  1.34356  1.34144  1.34260   
9998  07.12.2004 20:00:00.000  1.34266  1.34308  1.34200  1.34249   
9999  07.12.2004 21:00:00.000  1.34214  1.34290  1.34134  1.34162   

            volume   EMA_150  
9995  1.356140e+08  1.334397  
9996  1.473817e+08  1.334501  
9997  1.078832e+08  1.334608  
9998  6.438960e+07  1.334713  
9999  3.087270e+07  1.334804  


## Trend detection

## Breakout

In [4]:
def is_pivot(candle,window):
  '''
  Checks whether the current candle is a pivot candle or not.
  A pivot candle is one that is either higher than or lower than all its neighbours in some defined neighbourhood.

  Returns :
    3 - Mixed signal
    2 - A pivot low candle (support)
    1 - A pivot high candle (resistance)
    0 - No significance'''
  if candle-window<0 or candle+window>=len(df):
    return 0
  pivot_high=1
  pivot_low=2

  for i in range(candle-window,candle+window+1):
    if df.iloc[candle].low>df.iloc[i].low:
      pivot_low=0
    if df.iloc[candle].high<df.iloc[i].high:
      pivot_high=0

  if pivot_high and pivot_low:
    return 3
  elif pivot_high:
    return pivot_high
  elif pivot_low:
    return pivot_low
  else:
    return 0

In [5]:
window=10
df['is_pivot']= df.apply(lambda x : is_pivot(x.name,window),axis=1)

In [6]:
def point_pos(x):
  if x['is_pivot']==2:
    return x['low']-1e-3
  elif x['is_pivot']==1:
    return x['high']+1e-3
  return np.nan

df['pointpos'] = df.apply(lambda row: point_pos(row), axis=1)

In [13]:
dfpl = df[7000:8000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [14]:
def detect_level(candle,back_candles,window):
  '''
  Detects support and resistance levels according to pivot points

  Warning : the window argument must be greater than window used for checking pivots.'''
  # constraint for candle window
  if (candle<=back_candles+window) or (candle+window+1>=len(df)):
    return 0

  local_df=df.iloc[candle-back_candles-window:candle+window]

  highs=local_df[local_df['is_pivot']==1].high.tail(3).values
  lows=local_df[local_df['is_pivot']==2].low.tail(3).values
  levelbreak=0
  zone_width=0.001    # for EUR/USD

  if len(lows)==3:
    support_condition=True
    mean_lows=lows.mean()
    for low in lows:
      # check for alignment of lows, validates if it is a correct signal
      if abs(low-mean_lows)>zone_width:
        support_condition=False   # not sufficiently aligned
        break
    # check for bearish breakout
    if support_condition and (mean_lows - df.loc[candle].close)>zone_width*2:
      levelbreak = 1

  if len(highs)==3:
    resistance_condition=True
    mean_highs=highs.mean()
    for high in highs:
      if abs(high-mean_highs)>zone_width:
        resistance_condition=False
        break
    # bullish breakout
    if resistance_condition and (mean_highs - df.loc[candle].close)>zone_width*2:
      levelbreak = 2

  return levelbreak



In [15]:
df['pattern_detected'] =df.apply(lambda row: detect_level(row.name, back_candles=60, window=11), axis=1)

In [17]:
df[df['pattern_detected']!=0]

,Gmt time,open,high,low,close,volume,EMA_150,is_pivot,pointpos,pattern_detected
295,21.05.2003 04:00:00.000,1.17166,1.17207,1.17128,1.17163,2.730100e+07,1.156897,0,NaN,2
296,21.05.2003 05:00:00.000,1.17130,1.17183,1.17103,1.17139,2.471440e+07,1.157089,0,NaN,2
297,21.05.2003 06:00:00.000,1.17101,1.17168,1.17017,1.17046,2.752590e+07,1.157266,0,NaN,2
298,21.05.2003 07:00:00.000,1.17044,1.17285,1.16973,1.17114,2.843050e+07,1.157450,0,NaN,2
299,21.05.2003 08:00:00.000,1.17092,1.17180,1.17041,1.17132,2.657030e+07,1.157633,0,NaN,2
...,...,...,...,...,...,...,...,...,...,...
7879,05.08.2004 12:00:00.000,1.20442,1.20535,1.20189,1.20271,3.408560e+07,1.206160,2,1.20089,2
7880,05.08.2004 13:00:00.000,1.20263,1.20394,1.20248,1.20303,3.157870e+07,1.206119,0,NaN,2
7881,05.08.2004 14:00:00.000,1.20320,1.20507,1.20283,1.20477,3.277110e+07,1.206101,0,NaN,2
7882,05.08.2004 15:00:00.000,1.20443,1.20564,1.20417,1.20490,3.337810e+07,1.206085,0,NaN,2
